needed to reload modules, otherwise you will have nan errors

In [1]:
#%load_ext autoreload
#%autoreload 2

### Data

In [2]:
from data_process_and_get_mask import create_mask
from get_data import pbmc_train, pbmc_val

tf_list = ['IRF1', 'IRF2', 'IRF2BPL', 'IRF3', 'IRF4', 'IRF5', 'IRF6', 'IRF7',
       'IRF8', 'IRF9', 'STAT1', 'STAT2', 'STAT3', 'STAT4', 'STAT5A',
       'STAT5B','STAT6','GTF2I','RUNX1', 'MYOD1', 'SOX2','KLF4', 'THAP11','ADNP2', 'AEBP1','AHRR', 'ALX4','APBB1','APEX1', 'ARHGAP35','ARID1A']

#['IRF1', 'IRF2', 'IRF2BPL', 'IRF3', 'IRF4', 'IRF5', 'IRF6', 'IRF7', 'IRF8', 'IRF9', 'STAT1', 'STAT2', 'STAT3', 'STAT4', 'STAT5A','STAT5B', 'STAT6']
#["GTF2I","GTF3A","NRF1","ELF1","STAT1","STAT2","IRF9","STAT3","STAT4","STAT5A","STAT5B","IRF3","IRF7","IRF1","IRF5","IRF8"]

mask, train, valid = create_mask(pbmc_train, pbmc_val, tf_list)
print("Printing mask:")
print(mask)
print("\nPrinting train:")
print(train)
print("\nPrinting valid:")
print(valid)

Printing mask:
shape: (471, 32)
┌────────┬────────┬────────┬───────┬───┬────────┬───────┬───────┬───────────────┐
│ target ┆ STAT5A ┆ STAT5B ┆ STAT4 ┆ … ┆ ARID1A ┆ ADNP2 ┆ APBB1 ┆ unannotated_1 │
│ ---    ┆ ---    ┆ ---    ┆ ---   ┆   ┆ ---    ┆ ---   ┆ ---   ┆ ---           │
│ str    ┆ f64    ┆ f64    ┆ f64   ┆   ┆ f64    ┆ f64   ┆ f64   ┆ i32           │
╞════════╪════════╪════════╪═══════╪═══╪════════╪═══════╪═══════╪═══════════════╡
│ IL2    ┆ 1.0    ┆ 1.0    ┆ 0.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ IRF1   ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ CCND1  ┆ 1.0    ┆ 1.0    ┆ 0.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ PRF1   ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ IFNG   ┆ 1.0    ┆ 1.0    ┆ 1.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 1             │
│ …      ┆ …      ┆ …      ┆ …     ┆ … ┆ …      ┆ …     ┆ …     ┆ …             │
│ ZNF181 ┆ 0.0    ┆ 0.0    ┆ 0.0   ┆ … ┆ 0.0    ┆ 0.0   ┆ 0.0   ┆ 

In [3]:
# model training
#vega, vega_losses, vega_klds, vega_mses = trainVEGA(vega, train, epochs = 20, beta = 0.0001) #takes about 2 mins on GPU # epoch 100

now use the common training functions

In [4]:
mask

target,STAT5A,STAT5B,STAT4,IRF1,STAT3,RUNX1,GTF2I,IRF2,STAT1,IRF8,STAT6,APEX1,IRF4,MYOD1,ARHGAP35,IRF3,IRF5,ALX4,STAT2,IRF7,KLF4,IRF9,AHRR,AEBP1,SOX2,THAP11,IRF6,ARID1A,ADNP2,APBB1,unannotated_1
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32
"""IL2""",1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""IRF1""",1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""CCND1""",1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
"""PRF1""",1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""IFNG""",1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZNF181""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""ZNF230""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
"""ZNF302""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
from training.training import run_vega_model
run_vega_model(
    model_type="vega",
    train_data=train,
    valid_data=valid,
    mask_df=mask,
    epochs=30,
    N=10,
    cond="control"
)

epoch:  0  train_loss:  121.63222   valid_loss:  102.3654
epoch:  10  train_loss:  63.31294   valid_loss:  47.911713
epoch:  20  train_loss:  54.895252   valid_loss:  41.293972
epoch:  0  train_loss:  118.466286   valid_loss:  98.02734
epoch:  10  train_loss:  62.935814   valid_loss:  47.67235
epoch:  20  train_loss:  53.9548   valid_loss:  41.97272
epoch:  0  train_loss:  117.88331   valid_loss:  96.87498
epoch:  10  train_loss:  62.865475   valid_loss:  46.66238
epoch:  20  train_loss:  54.215176   valid_loss:  42.721836
epoch:  0  train_loss:  121.65493   valid_loss:  100.93554
epoch:  10  train_loss:  63.101658   valid_loss:  47.292202
epoch:  20  train_loss:  54.130653   valid_loss:  41.82671
epoch:  0  train_loss:  121.7331   valid_loss:  103.845825
epoch:  10  train_loss:  63.27718   valid_loss:  48.57218
epoch:  20  train_loss:  55.03451   valid_loss:  42.18485
epoch:  0  train_loss:  121.53536   valid_loss:  103.879166
epoch:  10  train_loss:  64.460495   valid_loss:  48.35404

(VEGA(
   (decoder): DecoderBayes(
     (sparse_layer): BayesianSparseLayer()
   )
   (encoder): Encoder(
     (encoder): Sequential(
       (0): Linear(in_features=471, out_features=800, bias=True)
       (1): ReLU()
       (2): Dropout(p=0.3, inplace=False)
       (3): Linear(in_features=800, out_features=800, bias=True)
       (4): ReLU()
       (5): Dropout(p=0.3, inplace=False)
     )
     (mu): Sequential(
       (0): Linear(in_features=800, out_features=31, bias=True)
       (1): Dropout(p=0.5, inplace=False)
     )
     (sigma): Sequential(
       (0): Linear(in_features=800, out_features=31, bias=True)
       (1): Dropout(p=0.5, inplace=False)
     )
   )
 ),
           STAT5A    STAT5B     STAT4      IRF1     STAT3     RUNX1     GTF2I  \
 IL2     0.000164 -0.001575  0.000000  0.000000  0.000997 -0.000494  0.000000   
 IRF1   -0.004038  0.001053  0.006667 -0.002780 -0.004636  0.000000  0.000000   
 CCND1   0.001617 -0.000472  0.000000  0.001623  0.000513  0.000000  0.003482   